# NumPy - Cấu trúc nội bộ và thao tác mảng nâng cao

## Nhập thư viện

In [1]:
import numpy as np

## Tạo dữ liệu ngẫu nhiên bằng `random()`

In [2]:
rng = np.random.default_rng(seed=12345)

## A.1 Cấu trúc nội bộ của đối tượng ndarray

`ndarray` trong NumPy là một cấu trúc dữ liệu mảng đa chiều được xây dựng trên một khối bộ nhớ liên tục chứa các giá trị có cùng kiểu dữ liệu (dtype).

Một mảng NumPy bao gồm:

* Con trỏ dữ liệu – trỏ đến vùng nhớ chứa dữ liệu thô.

* Kiểu dữ liệu (dtype) – quy định mỗi phần tử là kiểu gì (int, float, bool, v.v.).

* Kích thước (shape) – là một tuple biểu diễn số chiều và số phần tử ở mỗi chiều.

* Strides – số byte cần bước qua để đến phần tử tiếp theo theo từng trục.

Ví dụ: Mảng 10x5 sẽ có kích thước (10,5)

In [3]:
np.ones((10, 5)).shape

(10, 5)

### Phân cấp kiểu dữ liệu trong NumPy

NumPy có hệ thống kế thừa giữa các kiểu dữ liệu:

*   np.integer bao gồm các kiểu số nguyên như int8, int16, uint32...
*   np.floating bao gồm các kiểu số thực như float16, float32, float64.

Cả hai đều kế thừa từ lớp cha np.number.

Kiểm tra kiểu dữ liệu bằng:

In [4]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

**Hệ thống phân cấp kiểu dữ liệu NumPy**


Ta có thể kiểm tra xem một mảng có chứa số nguyên, số dấu phẩy động, chuỗi hay đối tượng Python hay không bằng các siêu lớp, chẳng hạn như np.integer và np.floating, có thể được sử dụng với hàm `np.issubdtype`

In [5]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)

True

In [6]:
np.issubdtype(floats.dtype, np.floating)

True

Bạn có thể xem tất cả các lớp cha của một kiểu dữ liệu cụ thể bằng cách gọi phương thức `mro`

In [7]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [8]:
np.issubdtype(ints.dtype, np.number)

True

## A.2 Thao tác mảng nâng cao

### 1. Thay đổi hình dạng mảng (Reshaping)



*   Dùng `.reshape()` để chuyển mảng sang kích thước mới mà không cần sao chép dữ liệu.
*   Có thể truyền -1 để NumPy tự suy ra kích thước còn lại.
*   Ngược lại, `.ravel()` hoặc `.flatten()` dùng để làm phẳng (1 chiều):
    *   `ravel()` tạo view (không copy dữ liệu nếu có thể).
    * `flatten()` luôn tạo bản sao mới.




Trong nhiều trường hợp, ta có thể chuyển đổi một mảng từ hình dạng này sang hình dạng khác mà không cần sao chép bất kỳ dữ liệu nào. Để làm điều này, hãy truyền một bộ dữ liệu biểu thị hình dạng mới vào phương thức thể hiện mảng định hình lại.

In [9]:
arr = np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [10]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

Mảng đa chiều cũng có thể được định hình lại:

In [11]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

Một trong các chiều được truyền có thể là -1

In [12]:
arr = np.arange(15)
arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

Vì thuộc tính `shape` của mảng là một `tuple` nên nó cũng có thể được truyền vào `reshape`:

In [13]:
other_arr = np.ones((3, 5))
other_arr.shape

(3, 5)

In [14]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

Hoạt động ngược lại của việc reshape từ một chiều sang một chiều cao hơn thường được gọi là `flattening` hoặc `raveling`

In [15]:
arr = np.arange(15).reshape((5, 3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [16]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

`ravel` không tạo ra bản sao của các giá trị cơ bản nếu các giá trị trong kết quả nằm liền kề trong mảng gốc.

Phương thức `flatten` hoạt động giống như ravel ngoại trừ việc nó luôn trả về bản sao của dữ liệu:

In [17]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

### 2. Thứ tự bộ nhớ: C (row-major) và Fortran (column-major)

Mảng NumPy mặc định lưu theo **C order** (theo hàng).
`order='F'` sẽ lưu theo **Fortran order** (theo cột).
Ảnh hưởng đến hiệu năng khi tính toán và reshape.

Các hàm như `reshape` và `ravel` chấp nhận một đối số thứ tự cho biết thứ tự sử dụng dữ liệu trong mảng. Đối số này thường được đặt thành 'C' hoặc 'F' trong hầu hết các trường hợp (cũng có các tùy chọn ít được sử dụng hơn là 'A' và 'K').

In [18]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [20]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

Sự khác biệt chính giữa thứ tự C và FORTRAN là cách
duyệt các chiều:

* **Thứ tự C/hàng chính**: Duyệt các chiều cao hơn trước (ví dụ: trục 1 trước khi tiến lên trục 0).
* **Thứ tự FORTRAN/cột chính**:
Duyệt các chiều cao hơn sau cùng (ví dụ: trục 0 trước khi tiến lên trục 1).

### 3. Nối và tách mảng (Concatenation & Splitting)

* `np.concatenate()` dùng để ghép nhiều mảng lại với nhau.

* `np.vstack()` và `np.hstack()` là các hàm tiện dụng cho việc nối theo hàng hoặc cột.

* `np.split(arr, [1,3])` tách mảng tại các chỉ số đã cho.

Ngoài ra có các đối tượng đặc biệt:

* `np.r_[]` – nối theo chiều dọc (row).

* `np.c_[]` – nối theo chiều ngang (column).

`numpy.concatenate` lấy một chuỗi (bộ, danh sách, v.v.) các mảng và nối chúng theo thứ tự dọc theo trục đầu vào:

In [21]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [22]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

`vstack` và `hstack`, dành cho các kiểu ghép mảng thông dụng. Các phép toán trước đó có thể được biểu diễn như sau:

In [23]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [24]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

`split` chia một mảng thành nhiều mảng dọc theo một trục:

In [25]:
arr = rng.standard_normal((5, 2))
arr

array([[-1.42382504,  1.26372846],
       [-0.87066174, -0.25917323],
       [-0.07534331, -0.74088465],
       [-1.3677927 ,  0.6488928 ],
       [ 0.36105811, -1.95286306]])

In [26]:
first, second, third = np.split(arr, [1, 3])
first

array([[-1.42382504,  1.26372846]])

In [27]:
second

array([[-0.87066174, -0.25917323],
       [-0.07534331, -0.74088465]])

In [28]:
third

array([[-1.3677927 ,  0.6488928 ],
       [ 0.36105811, -1.95286306]])

Giá trị [1, 3] được truyền vào `np.split` biểu thị các chỉ số để chia mảng
thành các phần.

| Function | Description |
|-----------|-------------|
| `concatenate` | Most general function, concatenate collection of arrays along one axis |
| `vstack`, `row_stack` | Stack arrays by rows (along axis 0) |
| `hstack` | Stack arrays by columns (along axis 1) |
| `column_stack` | Like `hstack`, but convert 1D arrays to 2D column vectors first |
| `dstack` | Stack arrays by “depth” (along axis 2) |
| `split` | Split array at passed locations along a particular axis |
| `hsplit` / `vsplit` | Convenience functions for splitting on axis 0 and 1, respectively |


### 4. Trình trợ giúp xếp chồng: r_ và c_

Có hai đối tượng đặc biệt trong NumPy, *r_* và *c_*, giúp việc xếp chồng các mảng trở nên ngắn gọn hơn:

In [29]:
arr = np.arange(6)
arr1 = arr.reshape((3, 2))
arr2 = rng.standard_normal((3, 2))
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [ 2.34740965,  0.96849691],
       [-0.75938718,  0.90219827],
       [-0.46695317, -0.06068952]])

In [30]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [ 2.34740965,  0.96849691,  3.        ],
       [-0.75938718,  0.90219827,  4.        ],
       [-0.46695317, -0.06068952,  5.        ]])

Ngoài ra, chúng có thể dịch các lát cắt thành mảng:

In [31]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

### 5. Lặp lại phần tử (Repeat & Tile)

`repeat` sao chép từng phần tử trong một mảng nhiều lần, tạo ra một mảng lớn hơn:

In [32]:
arr = np.arange(3)
arr

array([0, 1, 2])

In [33]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

Nếu bạn truyền một mảng số nguyên, mỗi phần tử có thể được lặp lại với số lần khác nhau:

In [34]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

Mảng đa chiều có thể có các phần tử được lặp lại dọc theo một trục cụ thể:

In [35]:
arr = rng.standard_normal((2, 2))
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [36]:
arr.repeat(2, axis=0)

array([[ 0.78884434, -1.25666813],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899]])

Tương tự, ta có thể truyền một mảng số nguyên khi lặp lại một mảng đa chiều để lặp lại một lát cắt nhất định với số lần khác nhau:

In [37]:
arr.repeat([2, 3], axis=0)

array([[ 0.78884434, -1.25666813],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899]])

In [38]:
arr.repeat([2, 3], axis=1)

array([[ 0.78884434,  0.78884434, -1.25666813, -1.25666813, -1.25666813],
       [ 0.57585751,  0.57585751,  1.39897899,  1.39897899,  1.39897899]])

`np.tile()` lặp toàn bộ mảng theo hàng hoặc cột.

In [39]:
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [40]:
np.tile(arr, 2)

array([[ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899]])

Đối số thứ hai là số lần lặp (số “ô gạch” – tiles).
Khi đối số này là một giá trị vô hướng (scalar), việc lặp (tiling) sẽ được thực hiện theo từng hàng (row by row) thay vì theo cột.

Ngoài ra, đối số thứ hai của hàm `tile` cũng có thể là một tuple, dùng để chỉ định cách bố trí (layout) của các lần lặp — tức là số lần lặp theo chiều hàng và chiều cột tương ứng.

In [41]:
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [42]:
np.tile(arr, (2, 1))

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [43]:
np.tile(arr, (3, 2))

array([[ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899]])

### 6. Truy xuất nâng cao (Take & Put)

In [44]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

`take()` chọn phần tử theo chỉ số:

In [45]:
arr.take(inds)

array([700, 100, 200, 600])

In [46]:
arr.put(inds, 42)
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [47]:
arr.put(inds, [40, 41, 42, 43])
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

`put()` gán giá trị theo chỉ số (làm việc trên mảng phẳng 1D):

In [48]:
inds = [2, 0, 2, 1]
arr = rng.standard_normal((2, 4))
arr

array([[ 1.32229806, -0.29969852,  0.90291934, -1.62158273],
       [-0.15818926,  0.44948393, -1.34360107, -0.08168759]])

In [49]:
arr.take(inds, axis=1)

array([[ 0.90291934,  1.32229806,  0.90291934, -0.29969852],
       [-1.34360107, -0.15818926, -1.34360107,  0.44948393]])

Hàm `put` không chấp nhận đối số axis, mà thay vào đó chỉ đánh chỉ số (indexing) trên phiên bản phẳng một chiều (flattened, theo thứ tự C – C order) của mảng.

Vì vậy, khi bạn cần gán giá trị cho các phần tử bằng một mảng chỉ số (index array) trên các trục khác (không phải dạng phẳng), thì tốt nhất nên sử dụng cú pháp truy cập bằng dấu ngoặc vuông [] để đảm bảo đúng vị trí và cấu trúc của dữ liệu.